### The following code could be used to calibrate model outputs and to investigate if model calibration improved performance compared to the baseline unclibrated model

In [ ]:
#declare gpu use

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [ ]:
#clear warnings and session

import warnings 
warnings.filterwarnings('ignore',category=FutureWarning) 
import tensorflow as tf
from tensorflow.keras import backend as K
K.clear_session()

In [ ]:
#import other libraries

import time
import itertools
from itertools import cycle
from matplotlib import pyplot
from sklearn import metrics
from sklearn.calibration import calibration_curve
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, brier_score_loss
from numpy import sqrt
from numpy import argmax
import numpy as np
from scipy import interp
from numpy import genfromtxt
import pandas as pd
import math
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications import VGG16, DenseNet121, InceptionV3
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from sklearn.metrics import roc_curve, auc,  precision_recall_curve, average_precision_score, matthews_corrcoef
from sklearn.metrics import f1_score, cohen_kappa_score, precision_score, recall_score, classification_report, log_loss, confusion_matrix, accuracy_score 
from sklearn.utils import class_weight
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import ml_insights as mli
from betacal import BetaCalibration
print(mli.__version__)

In [ ]:
#custom functions to generate reliability diagram

def calc_bins(y_test, preds):
    num_bins = 10
    bins = np.linspace(0.1, 1, num_bins)
    binned = np.digitize(preds, bins)  
    bin_accs = np.zeros(num_bins)
    bin_confs = np.zeros(num_bins)
    bin_sizes = np.zeros(num_bins)

    for bin in range(num_bins):
        bin_sizes[bin] = len(preds[binned == bin])
    if bin_sizes[bin] > 0:
        bin_accs[bin] = (y_test[binned==bin]).sum() / bin_sizes[bin]
        bin_confs[bin] = (preds[binned==bin]).sum() / bin_sizes[bin]

    return bins, binned, bin_accs, bin_confs, bin_sizes

In [ ]:
# get ECE and MCE metrics

def get_metrics(y_test, preds):
    ECE = 0
    MCE = 0
    bins, _, bin_accs, bin_confs, bin_sizes = calc_bins(y_test, preds)
    for i in range(len(bins)):
        abs_conf_dif = abs(bin_accs[i] - bin_confs[i])
        ECE += (bin_sizes[i] / sum(bin_sizes)) * abs_conf_dif
        MCE = max(MCE, abs_conf_dif)
    return ECE,MCE

In [ ]:
# draw reliability diagram

def draw_reliability_graph(y_test, preds):
    ECE, MCE = get_metrics(y_test, preds)
    bins, _, bin_accs, _, _ = calc_bins(y_test, preds)
    fig = plt.figure(figsize=(15, 10), dpi=400)
    ax = fig.gca()
    # x/y limits
    ax.set_xlim(0, 1.05)
    ax.set_ylim(0, 1)
    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.tick_params(axis='both', which='minor', labelsize=15)
    # x/y labels
    plt.xlabel('Prediction', fontsize=20)
    plt.ylabel('Truth', fontsize=20)
    # Create grid
    ax.set_axisbelow(True) 
    ax.grid(color='gray', linestyle='dashed')
    # Error bars
    plt.bar(bins, bins,width=0.1,alpha=0.3,edgecolor='black', color='orange', hatch='\\')
    # Draw bars and identity line
    plt.bar(bins, bin_accs, width=0.1, alpha=1, 
          edgecolor='black', color='red') # b before 
    plt.plot([0,1],[0,1], '--', color='gray', linewidth=2)
    # Equally spaced axes
    plt.gca().set_aspect('equal', adjustable='box')
    # ECE and MCE legend
    ECE_patch = mpatches.Patch(color='blue',  #green before
                             label='ECE = {:.2f}%'.format(ECE*100))
    MCE_patch = mpatches.Patch(color='green', 
                                label='MCE = {:.2f}%'.format(MCE*100))
    plt.legend(handles=[ECE_patch, MCE_patch],prop={'size': 15})  


In [ ]:
# Plot confusion matrix

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    print(cm)
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# compute performance metrics

def matrix_metrix(real_values,pred_values,beta):
    CM = confusion_matrix(real_values,pred_values)
    TN = CM[0][0]
    FN = CM[1][0] 
    TP = CM[1][1]
    FP = CM[0][1]
    Population = TN+FN+TP+FP
    Kappa = 2 * (TP * TN - FN * FP) / (TP * FN + TP * FP + 2 * TP * TN + FN**2 + FN * TN + FP**2 + FP * TN)
    Prevalence = round( (TP+FP) / Population,2)
    Accuracy   = round( (TP+TN) / Population,4)
    Precision  = round( TP / (TP+FP),4 )
    NPV        = round( TN / (TN+FN),4 )
    FDR        = round( FP / (TP+FP),4 )
    FOR        = round( FN / (TN+FN),4 ) 
    check_Pos  = Precision + FDR
    check_Neg  = NPV + FOR
    Recall     = round( TP / (TP+FN),4 )
    FPR        = round( FP / (TN+FP),4 )
    FNR        = round( FN / (TP+FN),4 )
    TNR        = round( TN / (TN+FP),4 ) 
    check_Pos2 = Recall + FNR
    check_Neg2 = FPR + TNR
    LRPos      = round( Recall/FPR,4 ) 
    LRNeg      = round( FNR / TNR ,4 )
    DOR        = round( LRPos/LRNeg)
    F1         = round ( 2 * ((Precision*Recall)/(Precision+Recall)),4)
    FBeta      = round ( (1+beta**2)*((Precision*Recall)/((beta**2 * Precision)+ Recall)) ,4)
    MCC        = round ( ((TP*TN)-(FP*FN))/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))  ,4)
    BM         = Recall+TNR-1
    MK         = Precision+NPV-1
    mat_met = pd.DataFrame({'Metric':['TP','TN','FP','FN','Prevalence','Accuracy','Precision','NPV','FDR','FOR','check_Pos','check_Neg','Recall','FPR','FNR','TNR','check_Pos2','check_Neg2','LR+','LR-','DOR','F1','FBeta','MCC','BM','MK','Kappa'],     'Value':[TP,TN,FP,FN,Prevalence,Accuracy,Precision,NPV,FDR,FOR,check_Pos,check_Neg,Recall,FPR,FNR,TNR,check_Pos2,check_Neg2,LRPos,LRNeg,DOR,F1,FBeta,MCC,BM,MK, Kappa]})
    return (mat_met)

In [ ]:
# load data
'''
Use the following to load data of all degrees of imbalances for the respective image modalities
'''
img_width, img_height = 256,256
train_data_dir = "data/train"
test_data_dir = "data/test"
epochs = 32 
batch_size = 32 
num_classes = 2
input_shape = (img_width, img_height, 3)
model_input = Input(shape=input_shape)
print(model_input) 

In [ ]:
#define data generators

datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.1) #90/10 train/val split

train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        seed=42,
        batch_size=batch_size, 
        class_mode='categorical', 
        subset = 'training')

validation_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        seed=42,
        batch_size=batch_size, 
        class_mode='categorical', 
        subset = 'validation')

test_generator = datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size, 
        class_mode='categorical', 
        shuffle = False)

#identify the number of samples
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

#check the class indices
print(train_generator.class_indices)
print(validation_generator.class_indices)
print(test_generator.class_indices)

#true labels
Y_test=test_generator.classes
print(Y_test.shape)

#convert test labels to categorical
Y_test1=to_categorical(Y_test, num_classes=num_classes, dtype='float32')
print(Y_test1.shape)

In [ ]:
#declare model architecture

vgg16_cnn = VGG16(include_top=False, weights='imagenet', 
                        input_tensor=model_input)
base_model_vgg16=Model(inputs=vgg16_cnn.input,
                        outputs=vgg16_cnn.get_layer('block5_conv3').output)
x = base_model_vgg16.output    
x = GlobalAveragePooling2D()(x)
logits = Dense(num_classes, 
                    activation='softmax', name='predictions')(x)
model_vgg16 = Model(inputs=base_model_vgg16.input, 
                    outputs=logits, 
                    name = 'vgg16_1')
model_vgg16.summary()

In [ ]:
#compile

sgd = SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)  
model_vgg16.compile(optimizer=sgd, 
                    loss='categorical_crossentropy', 
                    metrics=['accuracy']) 
filepath = 'model/' + model_vgg16.name + '.{epoch:02d}-{val_accuracy:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', 
                             verbose=1, 
                             save_weights_only=False, 
                             save_best_only=True, 
                             mode='min', 
                             save_freq='epoch')
earlyStopping = EarlyStopping(monitor='val_loss', 
                              patience=5, 
                              verbose=1, 
                              mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.5, 
                              patience=5,
                              verbose=1,
                              mode='min', 
                              min_lr=0.00001)
callbacks_list = [checkpoint, earlyStopping, reduce_lr]
t=time.time()

#reset generators
train_generator.reset()
validation_generator.reset()

#train the model
model_vgg16_history = model_vgg16.fit(train_generator, 
                                          steps_per_epoch=nb_train_samples // batch_size,
                                  epochs=epochs, validation_data=validation_generator,
                                  callbacks=callbacks_list, 
                                  validation_steps=nb_validation_samples // batch_size, 
                                  verbose=1)

print('Training time: %s' % (time.time()-t))

In [ ]:
# plot performance

N = 32 #change if early stopping
plt.style.use("ggplot")
plt.figure(figsize=(20,10), dpi=400)
plt.plot(np.arange(1, N+1), 
         model_vgg16_history.history["loss"], 'orange', label="train_loss")
plt.plot(np.arange(1, N+1), 
         model_vgg16_history.history["val_loss"], 'red', label="val_loss")
plt.plot(np.arange(1, N+1), 
          model_vgg16_history.history["accuracy"], 'blue', label="train_acc")
plt.plot(np.arange(1, N+1), 
         model_vgg16_history.history["val_accuracy"], 'green', label="val_acc")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower right")
plt.savefig("model_performance.png")

In [ ]:
# load and predict

vgg16_model = load_model('model.h5')
vgg16_model.summary()

#compile the model
sgd = SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)  
vgg16_model.compile(optimizer=sgd, 
                    loss='categorical_crossentropy', 
                    metrics=['accuracy']) 

#Generate predictions on the test data
test_generator.reset() 
custom_y_pred = vgg16_model.predict(test_generator,
                                    nb_test_samples // batch_size, 
                                    verbose=1)
custom_y_pred1_label = custom_y_pred.argmax(axis=-1)

#save the predictions
np.savetxt('y_pred.csv',
           custom_y_pred,fmt='%f',delimiter = ",")
np.savetxt('_test.csv',
           Y_test,fmt='%f',delimiter = ",")

In [ ]:
# if you want to save the predictions for each image along with the filenames to a CSV file

predicted_class_indices=np.argmax(custom_y_pred,axis=1)
print(predicted_class_indices)
labels = (test_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

#save the results to a CSV file
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":custom_y_pred1,
                      "Labels":predictions})
results.to_csv("results.csv",index=False)

In [ ]:
#we need the scores of only the positive abnormal class

custom_y_pred1 = custom_y_pred[:,1]

In [ ]:
#for default predictions, the threshold is 0.5, vary this value for optimal PR-based thresholds

custom_y_pred1_opt = np.where(custom_y_pred1 > 0.5, 1, 0) 
print(custom_y_pred1_opt)

In [ ]:
#plot reliability diagram

preds_calibrated = custom_y_pred1
draw_reliability_graph(Y_test, preds_calibrated)

In [ ]:
# we can also plot the calibration curves using the Scikit-learn package

fig = plt.figure(1, figsize=(15, 10), dpi=400)
ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
ax2 = plt.subplot2grid((3, 1), (2, 0))
ax1.tick_params(axis='both', which='major', labelsize=15)
ax1.tick_params(axis='both', which='minor', labelsize=15)
ax2.tick_params(axis='both', which='major', labelsize=15)
ax2.tick_params(axis='both', which='minor', labelsize=15)    
ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")    
frac_of_pos, mean_pred_value = calibration_curve(Y_test, custom_y_pred1, n_bins=10)
ax1.plot(mean_pred_value, frac_of_pos, "s-", label='Uncalibrated model', 
             color="red", linewidth=4)
ax1.set_ylabel("Truth", fontsize=20)
ax1.set_ylim([-0.05, 1.05])
ax1.legend(loc="upper right", prop={"size":20})    
ax2.hist(custom_y_pred1, range=(0, 1), bins=10, color = "blue", ec="red",
             label='Uncalibrated model', histtype="barstacked", lw=2)     
ax2.set_xlabel("Prediction", fontsize=20)
ax2.set_ylabel("Count", fontsize=20)

In [ ]:
# return total number of bins and the number of samples in each bin

def bin_total(y_true, y_prob, n_bins):
    bins = np.linspace(0., 1. + 1e-8, n_bins + 1)
    binids = np.digitize(y_prob, bins) - 1
    return np.bincount(binids, minlength=len(bins))
print(bin_total(Y_test,custom_y_pred1, n_bins=10))

In [ ]:
#print all metrics

mat_met = matrix_metrix(Y_test1.argmax(axis=-1),
                      custom_y_pred.argmax(axis=-1),
                      beta=0.4)
print (mat_met)



In [ ]:
# if you vary the optimal threshold value then use this
mat_met = matrix_metrix(Y_test1.argmax(axis=-1),
                      custom_y_pred1_opt,
                      beta=0.4)
print (mat_met)

# use these thresholded probabilites further to compute comfusion matrix

In [ ]:
#compute Brier score loss

print('The Brier Score Loss of the trained model is' , 
      round(brier_score_loss(Y_test,custom_y_pred[:,1]),4))

#compute Log loss

print('The Log Loss of the trained model is' , 
      round(log_loss(Y_test,custom_y_pred[:,1]),4))

In [ ]:
# print the confusion matrix

target_names = ['No-finding', 'TB'] #vary the labels for another imaging modality
print(classification_report(Y_test1.argmax(axis=-1),
                            custom_y_pred.argmax(axis=-1),
                            target_names=target_names, digits=4))

# Compute confusion matrix
cnf_matrix = confusion_matrix(Y_test1.argmax(axis=-1),
                              custom_y_pred.argmax(axis=-1))
np.set_printoptions(precision=5)
x_axis_labels = ['No-finding', 'TB'] 
y_axis_labels = ['No-finding', 'TB'] 
plt.figure(figsize=(10,10), dpi=400)
sns.set(font_scale=2)
b = sns.heatmap(cnf_matrix, annot=True, square = True, 
            cbar=False, cmap='Greens', 
            annot_kws={'size': 30},
            fmt='g', 
            xticklabels=x_axis_labels, 
            yticklabels=y_axis_labels)

In [ ]:
#plot the ROC curves with optimum threshold

fpr, tpr, thresholds = roc_curve(Y_test, 
                                 custom_y_pred[:,1])
auc_score=roc_auc_score(Y_test, custom_y_pred[:,1])
print(auc_score)

# calculate the g-mean for each threshold
gmeans = sqrt(tpr * (1-fpr))

# get the Youden's J statistic
J = tpr - fpr

# locate the index of the largest J statistic
ix_youden = argmax(J)

# locate the index of the largest g-means
ix_gmeans = argmax(gmeans)

#get the best threshold with G-means
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix_gmeans], gmeans[ix_gmeans]))

#get the best threshold with Youden's J statistic
print('Best Threshold=%f, Youden J statistic=%.3f' % (thresholds[ix_youden], J[ix_youden]))

# Plot all ROC curves
fig=plt.figure(figsize=(15,10), dpi=400)
ax = fig.add_subplot(1, 1, 1)
ax.set_facecolor('white')
major_ticks = np.arange(0.0, 1.1, 0.20) 
minor_ticks = np.arange(0.0, 1.1, 0.20)
ax.set_xticks(major_ticks)
ax.set_xticks(minor_ticks, minor=True)
ax.set_yticks(major_ticks)
ax.set_yticks(minor_ticks, minor=True)
plt.plot([0, 1], [0, 1], 'k--', lw=2, 
         label='No Skill')
plt.plot(fpr, tpr, 
         marker='.',
         markersize=12,
         markerfacecolor='green',
         linewidth=4,
         color='red',
         label='VGG-16')

#select the optimum point using G-means
plt.scatter(fpr[ix_gmeans], 
               tpr[ix_gmeans], 
               marker='X',           
               s=300, color='blue', 
               label='Optimal threshold')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel('False Positive Rate', fontsize=20)
plt.ylabel('True Positive Rate', fontsize=20)
plt.legend(loc="lower right", prop={"size":20})
plt.show()

In [ ]:
# find the optimal threshold on the PR curve that maximizes the F-score

precision, recall, thresholds = precision_recall_curve(Y_test, 
                                 custom_y_pred[:,1])
fscore = (2 * precision * recall) / (precision + recall)

#compute average precision
average_precision_base = average_precision_score(Y_test, 
                                 custom_y_pred[:,1])
print("The average precision value is", average_precision_base)

# area under the PR curve
print("The area under the PR curve is", metrics.auc(recall, precision))

# locate the index of the largest f score
ix = argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f' % (thresholds[ix], fscore[ix]))

# plot the PR curve for the model
no_skill = len(Y_test[Y_test==1]) / len(Y_test)
fig=plt.figure(figsize=(15,10), dpi=400)
ax = fig.add_subplot(1, 1, 1)
ax.set_facecolor('white')
major_ticks = np.arange(0.0, 1.1, 0.20) 
minor_ticks = np.arange(0.0, 1.1, 0.20)
ax.set_xticks(major_ticks)
ax.set_xticks(minor_ticks, minor=True)
ax.set_yticks(major_ticks)
ax.set_yticks(minor_ticks, minor=True)
pyplot.plot([0,1], [no_skill,no_skill], linestyle='--', label='No Skill')
pyplot.plot(recall, precision, marker='.', color='red', label='VGG-16')
pyplot.scatter(recall[ix], precision[ix], marker='X', s=300, color='blue', label='Optimal threshold')
# axis labels
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel('Recall', fontsize=20)
plt.ylabel('Precision', fontsize=20)
plt.legend(loc="lower right", prop={"size":20})
plt.show()


### Model Calibration: 
##### We observed from the histogram that the probabilities produced by the model tend to assume extreme values. Therefore, calibration techniqueswere used to try and fix these distortions. Platt scaling is a logistic regression-based calibration method where we essentially just perform logistic regression on the output of the DL model (y_pred) with respect to the true class labels (Y_test).

In [ ]:
# Fit Platt scaling 

lr = LogisticRegression(C=99999999999, solver='lbfgs')
# fit the model 
t=time.time()
lr.fit(custom_y_pred1.reshape(-1,1), Y_test)
print('Training time: %s' % (time.time()-t))
custom_y_pred1_platts1 = lr.predict_proba(custom_y_pred1.reshape(-1,1))
print('Prediction time: %s' % (time.time()-t))
custom_y_pred1_platts = lr.predict_proba(custom_y_pred1.reshape(-1,1))[:,1]

In [ ]:
# Plot reliability diagram

preds_calibrated = custom_y_pred1_platts
draw_reliability_graph(Y_test,preds_calibrated)

In [ ]:
# print performance metrics

mat_met = matrix_metrix(Y_test1.argmax(axis=-1),
                      custom_y_pred1_platts1.argmax(axis=-1),
                      beta=0.4)
print (mat_met)

In [ ]:
# When you obtain the optimal threshold from the PR curve, use it

custom_y_pred1_platts_opt= np.where(custom_y_pred1_platts > 0.6237, 1, 0) #this is just an example, use your optimal threshold
print(custom_y_pred1_platts_opt)

#compute performance metrics using this thresholded probability
mat_met = matrix_metrix(Y_test1.argmax(axis=-1),
                      custom_y_pred1_platts_opt,
                      beta=0.4)
print(mat_met)

# use these thresholded probabilites further to compute comfusion matrix

#### Use the codes for confusion matrix, ROC curves, and PR curves from before to plot these diagrams for the Platt-scaled probabilites. An example is given herewith. The same can be performed for all calibrated probabilites. 

In [ ]:
# plot confusion matrix

target_names = ['No-finding', 'TB'] 
print(classification_report(Y_test1.argmax(axis=-1),
                            custom_y_pred1_platts1.argmax(axis=-1),
                            target_names=target_names, digits=4))

# Compute confusion matrix
cnf_matrix = confusion_matrix(Y_test1.argmax(axis=-1),
                              custom_y_pred1_platts1.argmax(axis=-1))
np.set_printoptions(precision=5)

x_axis_labels = ['No-finding', 'TB']
y_axis_labels = ['No-finding', 'TB']
plt.figure(figsize=(10,10), dpi=400)
sns.set(font_scale=2)
b = sns.heatmap(cnf_matrix, annot=True, square = True, 
            cbar=False, cmap='Greens', 
            annot_kws={'size': 30},
            fmt='g', 
            xticklabels=x_axis_labels, 
            yticklabels=y_axis_labels)

In [ ]:
#compute Brier score loss

print('The Brier Score Loss of the trained model is' , 
      round(brier_score_loss(Y_test,custom_y_pred1_platts),4))

#compute Log loss

print('The Log Loss of the trained model is' , 
      round(log_loss(Y_test,custom_y_pred1_platts),4))

In [ ]:
# plot roc curve and find the optimum threshold

fpr, tpr, thresholds = roc_curve(Y_test, 
                                 custom_y_pred1_platts)

#compute area under the ROC curve
auc_score=roc_auc_score(Y_test, custom_y_pred1_platts)
print(auc_score)

# calculate the g-mean for each threshold
gmeans = sqrt(tpr * (1-fpr))

# get the Youden's J statistic
J = tpr - fpr

# locate the index of the largest J statistic
ix_youden = argmax(J)

# locate the index of the largest g-means
ix_gmeans = argmax(gmeans)

#get the best threshold with G-means
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix_gmeans], gmeans[ix_gmeans]))

#get the best threshold with Youden's J statistic
print('Best Threshold=%f, Youden J statistic=%.3f' % (thresholds[ix_youden], J[ix_youden]))

# Plot all ROC curves
fig=plt.figure(figsize=(15,10), dpi=400)
ax = fig.add_subplot(1, 1, 1)
ax.set_facecolor('white')
major_ticks = np.arange(0.0, 1.1, 0.20) 
minor_ticks = np.arange(0.0, 1.1, 0.20)
ax.set_xticks(major_ticks)
ax.set_xticks(minor_ticks, minor=True)
ax.set_yticks(major_ticks)
ax.set_yticks(minor_ticks, minor=True)
plt.plot([0, 1], [0, 1], 'k--', lw=2, 
         label='No Skill')
plt.plot(fpr, tpr, 
         marker='.',
         markersize=12,
         markerfacecolor='green',
         linewidth=4,
         color='red',
         label='Platt-scaling')

#select the optimum threshold using G-means
plt.scatter(fpr[ix_gmeans], 
               tpr[ix_gmeans], 
               marker='X',           
               s=300, color='blue', 
               label='Optimal threshold')

plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel('False Positive Rate', fontsize=20)
plt.ylabel('True Positive Rate', fontsize=20)
plt.legend(loc="lower right", prop={"size":20})
plt.show()

In [ ]:
# calculate pr curves curves

precision, recall, thresholds = precision_recall_curve(Y_test, 
                                 custom_y_pred1_platts)
fscore = (2 * precision * recall) / (precision + recall)

#compute average precision
average_precision_platt = average_precision_score(Y_test, 
                                 custom_y_pred1_platts)
print("The average precision value is", average_precision_platt)

# area under the PR curve
print("The area under the PR curve is", metrics.auc(recall, precision))

# locate the index of the largest f score
ix = argmax(fscore)

print('Best Threshold=%f, F-Score=%.3f' % (thresholds[ix], fscore[ix]))
# plot the PR curve for the model
no_skill = len(Y_test[Y_test==1]) / len(Y_test)
fig=plt.figure(figsize=(15,10), dpi=400)
ax = fig.add_subplot(1, 1, 1)
ax.set_facecolor('white')
major_ticks = np.arange(0.0, 1.1, 0.20) 
minor_ticks = np.arange(0.0, 1.1, 0.20)
ax.set_xticks(major_ticks)
ax.set_xticks(minor_ticks, minor=True)
ax.set_yticks(major_ticks)
ax.set_yticks(minor_ticks, minor=True)
pyplot.plot([0,1], [no_skill,no_skill], linestyle='--', label='No Skill')
pyplot.plot(recall, precision, marker='.', color='red', label='Platt-scaling')
pyplot.scatter(recall[ix], precision[ix], marker='X', s=300, color='blue', label='Optimal threshold')
# axis labels
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel('Recall', fontsize=20)
plt.ylabel('Precision', fontsize=20)
plt.legend(loc="lower right", prop={"size":20})
plt.show()

#### Beta calibration: Similar to Platt scaling with a couple of important improvements. It is a 3-parameter family of curves rather than 2-parameter Family of curves includes the line y=x (so it won't mess it up if it's already calibrated). 

##### Reference: Kull, M., Filho, T.S. & Flach, P.. (2017). Beta calibration: a well-founded and easily implemented improvement on logistic calibration for binary classifiers. Proceedings of the 20th International Conference on Artificial Intelligence and Statistics, in PMLR 54:623-631

In [ ]:
# Fit three-parameter beta calibration

bc = BetaCalibration()
t=time.time()
bc.fit(custom_y_pred1, Y_test)
print('Training time: %s' % (time.time()-t))
#perform beta calibration
t=time.time()
custom_y_pred1_beta = bc.predict(custom_y_pred1)
print('Prediction time: %s' % (time.time()-t))
custom_y_pred1_beta_1 = np.array(1-custom_y_pred1_beta)
custom_y_pred1_beta_2 = np.c_[custom_y_pred1_beta_1, custom_y_pred1_beta]

In [ ]:
# plot reliability diagram

preds_calibrated = custom_y_pred1_beta
draw_reliability_graph(Y_test, preds_calibrated)

In [ ]:
# performance metrics

mat_met = matrix_metrix(Y_test1.argmax(axis=-1),
                      custom_y_pred1_beta_2.argmax(axis=-1),
                      beta=0.4)
print (mat_met)

In [ ]:
# When you obtain the optimal threshold from the PR curve, use it

custom_y_pred1_beta_opt= np.where(custom_y_pred1_beta > 0.375305, 1, 0) #this is just an example, use your optimal threshold
print(custom_y_pred1_beta_opt)

#compute performance metrics using this thresholded probability
mat_met = matrix_metrix(Y_test1.argmax(axis=-1),
                      custom_y_pred1_beta_opt,
                      beta=0.4)
print(mat_met)

# use these thresholded probabilites further to compute comfusion matrix

In [ ]:
#compute Brier score loss

print('The Brier Score Loss of the Beta calibrated model is' , 
      round(brier_score_loss(Y_test,custom_y_pred1_beta),4))

#compute Log loss

print('The Log Loss of the Beta calibrated model is' , 
      round(log_loss(Y_test,custom_y_pred1_beta),4))


#### SplineCalib fits a cubic smoothing spline to the relationship between the uncalibrated scores and the calibrated probabilities. Smoothing splines strike a balance between fitting the points well and having a smooth function. SplineCalib uses a smoothed logistic function - so the fit to data is measured by likelihood (i.e. log-loss) and the smoothness refers to the integrated second derivative before the logistic transformation. There is a nuisance parameter that trades off smoothness for fit. At one extreme it will revert to standard logistic regression (i.e. Platt scaling) and at the other extreme it will be a very wiggly function that fits the data but does not generalize well. SplineCalib automatically fits the nuisance parameter (though this can be adjusted by the user). The resulting calibration function is not necessarily monotonic. (In some cases this may be beneficial).

##### Lucena, B. Spline-based Probability Calibration. https://arxiv.org/abs/1809.07751

In [ ]:
# adjust Spline paramters for a better fit suiting your problem

splinecalib = mli.SplineCalib(penalty='l2',
                              knot_sample_size=40,
                              cv_spline=5,
                              unity_prior=False,
                              unity_prior_weight=128)

t=time.time()
splinecalib.fit(custom_y_pred1, Y_test)
print('Training time: %s' % (time.time()-t))
custom_y_pred1_spline = splinecalib.predict(custom_y_pred1)
print('Prediction time: %s' % (time.time()-t))
custom_y_pred1_spline_1 = np.array(1-custom_y_pred1_spline)
custom_y_pred1_spline_2 = np.c_[custom_y_pred1_spline_1, custom_y_pred1_spline]

In [ ]:
# plot reliability diagram

preds_calibrated = custom_y_pred1_spline
draw_reliability_graph(Y_test, preds_calibrated)

In [ ]:
# performance metrics

mat_met = matrix_metrix(Y_test1.argmax(axis=-1),
                      custom_y_pred1_spline_2.argmax(axis=-1),
                      beta=0.4)
print (mat_met)

In [ ]:
# When you obtain the optimal threshold from the PR curve, use it

custom_y_pred1_spline_opt= np.where(custom_y_pred1_spline > 0.375305, 1, 0) #this is just an example, use your optimal threshold
print(custom_y_pred1_spline_opt)

#compute performance metrics using this thresholded probability
mat_met = matrix_metrix(Y_test1.argmax(axis=-1),
                      custom_y_pred1_spline_opt,
                      beta=0.4)
print(mat_met)

# use these thresholded probabilites further to compute comfusion matrix

In [ ]:
#compute Brier score loss

print('The Brier Score Loss of the Spline calibrated model is' , 
      round(brier_score_loss(Y_test,custom_y_pred1_spline),4))

#compute Log loss

print('The Log Loss of the Spline calibrated model is' , 
      round(log_loss(Y_test,custom_y_pred1_spline),4))

In [ ]:
#You can plot the calibration curves together using the Scikit learn calibration curve method

labels = []
fig=plt.figure(figsize=(15,10), dpi=400)
ax = fig.add_subplot(1, 1, 1)
ax.set_facecolor('white')
major_ticks = np.arange(0.0, 1.1, 0.10) 
minor_ticks = np.arange(0.0, 1.1, 0.10)
ax.set_xticks(major_ticks)
ax.set_xticks(minor_ticks, minor=True)
ax.set_yticks(major_ticks)
ax.set_yticks(minor_ticks, minor=True)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.tick_params(axis='both', which='minor', labelsize=15)
frac_of_positives_uncalibrated, pred_prob_uncalibrated = calibration_curve(Y_test,custom_y_pred1, n_bins=10)
plt.plot(pred_prob_uncalibrated,
         frac_of_positives_uncalibrated,
         linewidth=4,
         color='green')
labels.append('Uncalibrated')
frac_of_positives_platt, pred_prob_platt = calibration_curve(Y_test, custom_y_pred1_platts, n_bins=10)
plt.plot(pred_prob_platt,
         frac_of_positives_platt,
         linewidth=4,
         color='blue')
labels.append('Platt scaling')
frac_of_positives_beta, pred_prob_beta = calibration_curve(Y_test,custom_y_pred1_beta, n_bins=10)
plt.plot(pred_prob_beta,
          frac_of_positives_beta,
          linewidth=4,
          color='yellow')
labels.append('Beta calibration')
frac_of_positives_spline, pred_prob_spline = calibration_curve(Y_test,custom_y_pred1_spline, n_bins=10)
plt.plot(pred_prob_spline,
         frac_of_positives_spline,
         linewidth=4,
         color='red')
labels.append('Spline calibration')
plt.plot([0, 1], [0, 1], color='black', 
         linestyle='dashed', 
         linewidth = 1)
labels.append('Perfectly calibrated')
plt.legend(labels,loc="lower right", prop={"size":20})
xlabel = plt.xlabel("Prediction", fontsize=20)
ylabel = plt.ylabel("Truth", fontsize=20)
plt.show()

In [ ]:
# You can compare the PR curves of uncalibrated and calibrated curves and the optimal threshold values
# an example is shown herewith

precision1, recall1, thresholds1 = precision_recall_curve(Y_test, 
                                 custom_y_pred1_spline) # using spline-calibrated probabilities
precision2, recall2, thresholds2 = precision_recall_curve(Y_test, 
                                 custom_y_pred[:,1]) # using baseline uncalibrated probabilities
average_precision_spline = average_precision_score(Y_test, 
                                 custom_y_pred1_spline)
print("The average precision value of Spline calibration is", average_precision_spline)
average_precision_base = average_precision_score(Y_test, 
                                 custom_y_pred[:,1])
print("The average precision value of Baseline model is", average_precision_base)
fscore1 = (2 * precision1 * recall1) / (precision1 + recall1)
fscore2 = (2 * precision2 * recall2) / (precision2 + recall2)

# locate the index of the largest f score
ix1 = argmax(fscore1)
ix2 = argmax(fscore2)

#best threshold with the calibration method
print('Best Threshold=%f, F-Score=%.3f' % (thresholds1[ix1], fscore1[ix1]))

#best threshold with the baseline model
print('Best Threshold=%f, F-Score=%.3f' % (thresholds2[ix2], fscore2[ix2]))

# plot the PR curve for the model
no_skill = len(Y_test[Y_test==1]) / len(Y_test)
fig=plt.figure(figsize=(15,10), dpi=400)
ax = fig.add_subplot(1, 1, 1)
ax.set_facecolor('white')
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["left"].set_visible(False)
major_ticks = np.arange(0.0, 1.1, 0.20) 
minor_ticks = np.arange(0.0, 1.1, 0.20)
ax.set_xticks(major_ticks)
ax.set_xticks(minor_ticks, minor=True)
ax.set_yticks(major_ticks)
ax.set_yticks(minor_ticks, minor=True)
pyplot.plot([0,1], [no_skill,no_skill], linestyle='--', label='No Skill')
pyplot.plot(recall1, precision1, marker='.', color='black', label='Spline calibration') 
pyplot.plot(recall2, precision2, marker='.', color='red', label='Baseline')
pyplot.scatter(recall1[ix1], precision1[ix1], marker='o', 
               s=300, color='blue', label='Optimal-Spline calibration')
pyplot.scatter(recall2[ix2], precision2[ix2], marker='*', 
               s=400, color='yellow', label='Optimal-Baseline')
# axis labels
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel('Recall', fontsize=20)
plt.ylabel('Precision', fontsize=20)
plt.legend(loc="lower left", prop={"size":20})
plt.show()